In [21]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [23]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [26]:
import tensorflow as tf
print(tf.__version__)

2.18.0


In [ ]:
pip uninstall tensorflow

In [ ]:
pip install tensorflow

In [ ]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential

In [3]:
# Data Preparation for LSTM 
# Create a sample dataframe for testing
data = pd.DataFrame({
 'Month': pd.date_range(start='1/1/2000', periods=120, freq='M'),
 'Passengers': np.random.randint(100, 500, size=120) # Random passenger counts
})

C:\Users\sandh\AppData\Local\Temp\ipykernel_34860\2145340393.py:4: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  'Month': pd.date_range(start='1/1/2000', periods=120, freq='M'),


In [4]:
# Set 'Month' as the index
data.set_index('Month', inplace=True)

In [5]:
# Display the first few rows of the dataframe
print(data.head())

            Passengers
Month                 
2000-01-31         200
2000-02-29         385
2000-03-31         198
2000-04-30         492
2000-05-31         450


In [6]:
# Scale the data using MinMaxScaler (scales between 0 and 1)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data['Passengers'].values.reshape(-
1, 1))

In [8]:
# Create a function to prepare the data into sequences
def create_dataset(data, time_step=1):
 X, y = [], []
 for i in range(len(data) - time_step - 1):
     X.append(data[i:(i + time_step), 0])
     y.append(data[i + time_step, 0])
 return np.array(X), np.array(y)

In [9]:
# Define the time step (number of previous time steps to consider for each prediction)
time_step = 12

In [10]:
# Prepare the data
X, y = create_dataset(scaled_data, time_step)

In [11]:
# Reshape X to be in the shape [samples, time steps, features] for LSTM input
X = X.reshape(X.shape[0], X.shape[1], 1)

In [12]:
# Splitting the data into Training and Test Sets (80% Training, 20% Testing)
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
# Check the shapes of the prepared data
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(85, 12, 1) (22, 12, 1) (85,) (22,)


In [ ]:
### Building the LSTM Model

# Initialize the model
model = Sequential()

# Add the LSTM layer with 50 units and return sequences for the next LSTM layer
model.add(LSTM(units=50, return_sequences=True, 
input_shape=(X_train.shape[1], 1)))

# Add a Dropout layer to prevent overfitting
model.add(Dropout(0.2))

# Add another LSTM layer
model.add(LSTM(units=50, return_sequences=False))

# Add another Dropout layer
model.add(Dropout(0.2))

# Add the output layer with a single neuron for regression
model.add(Dense(units=1))

# Compile the model using Adam optimizer and MSE (Mean Squared Error) loss function
model.compile(optimizer=Adam(learning_rate=0.001), 
loss='mean_squared_error')

In [ ]:
# Train the model
model.fit(X, y, epochs=20, batch_size=32)

In [ ]:
# Model Evaluation 
from sklearn.metrics import mean_squared_error, mean_absolute_error
# Predictions
predicted = model.predict(X)

# Evaluate the model
rmse = np.sqrt(mean_squared_error(y, predicted))
mae = mean_absolute_error(y, predicted)

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")

In [ ]:
# Future Forecasting 
# Define sequence length for LSTM
sequence_length = 60 # Number of previous days the model looks at

# Predicting the next value (after the last available data)
future_sequence = scaled_data[-sequence_length:]
future_sequence = future_sequence.reshape(1, sequence_length, 1)
predicted_value = model.predict(future_sequence)
predicted_value = scaler.inverse_transform(predicted_value) # Rescale to original scale

print(f"Predicted value for the next day: {predicted_value}")

In [ ]:
# Number of days you want to forecast
forecast_days = 30 # Example: forecast next 30 days

# Prepare the last available sequence for prediction
last_sequence = scaled_data[-sequence_length:]
last_sequence = last_sequence.reshape(1, sequence_length, 1)

# Create a list to store the predictions
predicted_values = []

# Predict the next `forecast_days` days
for i in range(forecast_days):
 # Make the prediction
 next_day_prediction = model.predict(last_sequence)
 
 # Rescale the prediction back to original scale
 next_day_prediction =
scaler.inverse_transform(next_day_prediction)
 
 # Store the predicted value
 predicted_values.append(next_day_prediction[0][0])
 
 # Update the sequence to include the new prediction (for the next day's prediction)
 last_sequence = np.append(last_sequence[:, 1:, :], 
next_day_prediction.reshape(1, 1, 1), axis=1)

# Convert the predictions into a DataFrame for easier viewing
predicted_dates = pd.date_range(data.index[-1] + pd.Timedelta(days=1),
periods=forecast_days, freq='D')
predictions30_df = pd.DataFrame(predicted_values, 
index=predicted_dates, columns=['Predicted'])